# Module Imports

In [1]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Formatting

In [2]:
read_file = pd.read_excel ("30_01_2023_Mikala Furuto.xlsx")
read_file.to_csv ("30_01_2023_Mikala Furuto.csv", index = None, header=True)

mikala_all = pd.read_csv('30_01_2023_Mikala Furuto.csv')
mikala_all.drop_duplicates()
mikala_all = mikala_all[:-1]


mikala_numeric = mikala_all.drop(columns=['Date','Opponent','Unnamed: 1'])
mikala_numeric["Chances, % of conversion"] = np.where(mikala_numeric["Chances, % of conversion"].str.endswith('%'), mikala_numeric["Chances, % of conversion"].str.slice(stop=-1), '0')
mikala_numeric["Challenges won, %"] = np.where(mikala_numeric["Challenges won, %"].str.endswith('%'), mikala_numeric["Challenges won, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges in attack won, %"] = np.where(mikala_numeric["Challenges in attack won, %"].str.endswith('%'), mikala_numeric["Challenges in attack won, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges in defence won, %"] = np.where(mikala_numeric["Challenges in defence won, %"].str.endswith('%'), mikala_numeric["Challenges in defence won, %"].str.slice(stop=-1), '0')
mikala_numeric["Air challenges won, %"] = np.where(mikala_numeric["Air challenges won, %"].str.endswith('%'), mikala_numeric["Air challenges won, %"].str.slice(stop=-1), '0')
mikala_numeric["Successful dribbles, %"] = np.where(mikala_numeric["Successful dribbles, %"].str.endswith('%'), mikala_numeric["Successful dribbles, %"].str.slice(stop=-1), '0')
mikala_numeric["Tackles won, %"] = np.where(mikala_numeric["Tackles won, %"].str.endswith('%'), mikala_numeric["Tackles won, %"].str.slice(stop=-1), '0')

mikala_numeric = mikala_numeric.mask((mikala_numeric == '-'), 0)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] > mikala_numeric["Score"].str[-1], 100)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] == mikala_numeric["Score"].str[-1], 0)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] < mikala_numeric["Score"].str[-1], -100)


mikala_numeric = mikala_numeric.apply(pd.to_numeric, errors='ignore')


mikala_numeric


,Score,InStat Index,Position,Goals,Assists,Chances,Chances successful,"Chances, % of conversion",Сhances created,Fouls,...,Attacking challenges,"Challenges in attack won, %",Air challenges,"Air challenges won, %",Dribbles,"Successful dribbles, %",Tackles,"Tackles won, %",Ball interceptions,Free ball pick ups
0,-100,113,CD,0,0,0,0,0,0,0,...,3,67,2,100,1,100,7,57,14,8
1,0,130,CD,0,0,0,0,0,0,0,...,1,100,2,100,1,100,5,40,4,4
2,100,141,CD,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,8,2
3,100,146,CD,0,0,0,0,0,0,1,...,2,100,1,100,1,100,0,0,16,8
4,100,140,CD,0,0,0,0,0,0,0,...,2,100,4,100,0,0,1,100,24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,100,140,LM,0,0,0,0,0,0,0,...,3,33,1,0,3,33,0,0,0,6
58,100,170,LM,1,1,1,1,100,3,0,...,4,75,0,0,3,100,2,50,0,2
59,100,167,RM,0,0,0,0,0,0,0,...,7,57,1,0,5,60,6,100,5,3
60,100,184,RM,1,0,1,1,100,1,0,...,2,100,0,0,2,100,4,25,8,6


In [3]:
mikala_CD = mikala_numeric.loc[mikala_numeric['Position'] == 'CD']
mikala_CM = mikala_numeric.loc[mikala_numeric['Position'] == 'CM']
mikala_LD = mikala_numeric.loc[mikala_numeric['Position'] == 'LD']
mikala_LM = mikala_numeric.loc[mikala_numeric['Position'] == 'LM']
mikala_RD = mikala_numeric.loc[mikala_numeric['Position'] == 'RD']
mikala_RM = mikala_numeric.loc[mikala_numeric['Position'] == 'RM']


In [4]:
# mikala_numeric.corr(method='pearson').round(2)

In [5]:
mikcorr = mikala_numeric.corr(method='spearman').round(2)
# mikcorr.to_excel('Mikala_Correlation.xlsx', engine = 'xlsxwriter')

In [6]:
rounded_LM = mikala_LM.corr(method='spearman').round(2)

# Print Meaningful Correlations in Top 2 Positions

In [7]:
def important(Dataframe):
    Dataframe_important = Dataframe.iloc[:,0:2]
    Dataframe_important = Dataframe_important.loc[(Dataframe_important['Score'].abs() > 0.4) | (Dataframe_important['InStat Index'].abs() > 0.4)]
    return Dataframe_important

important(rounded_LM)

,Score,InStat Index
Score,1.00,0.29
InStat Index,0.29,1.00
Goals,0.26,0.55
Assists,0.17,0.44
Chances,0.29,0.50
Chances successful,0.26,0.55
"Chances, % of conversion",0.26,0.55
Сhances created,0.22,0.42
Shots,-0.12,0.62
Shots on target,0.14,0.64


In [8]:
rounded_CD = mikala_CD.corr(method='spearman').round(2)
important(rounded_CD)

,Score,InStat Index
Score,1.00,0.46
InStat Index,0.46,1.00
Ball recoveries in opponent's half,0.00,0.54
"Challenges won, %",0.20,0.53
"Challenges in defence won, %",0.12,0.62
Attacking challenges,-0.03,0.42
Tackles,-0.45,-0.35
Ball interceptions,0.45,0.33
